In [2]:
import random
import math
from csv import reader
import pandas as pd
import numpy as np
import random as rd
from statistics import mean
import sys
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from matplotlib import style

style.use('ggplot')
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [3]:
def TrueP(actual, predicted):
    count = 0

    for i in range(len(actual)):
        if actual[i] == 'Iris-setosa' and predicted[i] == 'Iris-setosa':
            count += 1
        if actual[i] == 'Iris-versicolor' and predicted[i] == 'Iris-versicolor':
            count += 1
        if actual[i] == 'Iris-virginica' and predicted[i] == 'Iris-virginica':
            count += 1
    return count


def TrueN(actual, predicted):
    count = 0
    for i in range(len(actual)):
        if actual[i] != 'Iris-setosa' and predicted[i] != 'Iris-setosa':
            count += 1
        if actual[i] != 'Iris-versicolor' and predicted[i] != 'Iris-versicolor':
            count += 1
        if actual[i] != 'Iris-virginica' and predicted[i] != 'Iris-virginica':
            count += 1
       
    return count


def FalseP(actual, predicted):
    count = 0

    for i in range(len(actual)):
        if actual[i] == 'Iris-setosa' and (predicted[i] == 'Iris-versicolor' or predicted[i] == 'Iris-virginica'):
            count += 1
        if actual[i] == 'Iris-versicolor' and (predicted[i] == 'Iris-setosa' or predicted[i] == 'Iris-virginica'):
            count += 1
        if actual[i] == 'Iris-virginica' and (predicted[i] == 'Iris-versicolor' or predicted[i] == 'Iris-setosa'):
            count += 1
    return count


def FalseN(actual, predicted):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 'Iris-setosa' and predicted[i] != 'Iris-setosa':
            count += 1
        if actual[i] == 'Iris-versicolor' and predicted[i] != 'Iris-versicolor':
            count += 1
        if actual[i] == 'Iris-virginica' and predicted[i] != 'Iris-virginica':
            count += 1        

    return count


def compute_mem_matrix(m, x, clusters, curr):
    res = []
    sum = 0
    for i in range(len(x)-1):
        U = 0

        if x[i] == curr[i]:
            U = 1
        else:
            ans = 0
            flag = True
            for c in clusters:
                if x[i] == clusters[c][i]:
                    U = 0
                    flag = False
                    break
                ans += pow((abs(x[i]-curr[i]) / abs(x[i]-clusters[c][i])), (2/(m-1)))
            if flag:
                U = 1/ans
                # print(U)
        res.append((U, x[i]))
        sum += U

    return [res, sum]


def compute_cluster_center(m, U_X):
    U = []
    X = []
    for i in U_X:
        U.append(i[0])
        X.append(i[1])
    num = 0
    den = 0
    for i in range(len(U)):
        #print(U[i]," ",X[i])
        num += (pow(U[i], m)*X[i])
        den += pow(U[i], m)
    c = num/den
    return c


def cMean(clusters, dataset, m):
    prev = {'Iris-setosa': clusters['Iris-setosa'], 'Iris-versicolor':clusters['Iris-versicolor'],'Iris-virginica': clusters['Iris-virginica']}
    final_cluster = {'Iris-setosa': [], 'Iris-versicolor':'Iris-versicolor',  'Iris-virginica': 'Iris-virginica'}
    arr = ['Iris-versicolor' for i in range(len(dataset))]
    epoch = 1
    for i in range(2000):
        cluster_Setosa = []
        cluster_Versicolor = []
        cluster_Virginica = []
        for i in range(len(dataset)):
            if dataset[i] == clusters['Iris-setosa']:
                arr[i] = 'Iris-setosa'
                ans = [(1, dataset[i][j]) for j in range(len(dataset[i]))]
                cluster_Setosa.append(ans)
                continue
            elif dataset[i] == clusters['Iris-versicolor']:
                arr[i] = 'Iris-versicolor'
                ans = [(1, dataset[i][j]) for j in range(len(dataset[i]))]
                cluster_Versicolor.append(ans)
                continue
            elif dataset[i] == clusters['Iris-virginica']:
                arr[i] = 'Iris-virginica'
                ans = [(1, dataset[i][j]) for j in range(len(dataset[i]))]
                cluster_Virginica.append(ans)
                continue
            c1 = compute_mem_matrix(m, dataset[i], clusters, clusters['Iris-setosa'])
            c2 = compute_mem_matrix(m, dataset[i], clusters, clusters['Iris-versicolor'])
            c3 = compute_mem_matrix(m, dataset[i], clusters, clusters['Iris-virginica'])
            if c1[1] > c2[1] and c1[1] > c3[1]:
                arr[i] = 'Iris-setosa'
                cluster_Setosa.append(c1[0])
            elif c2[1] > c3[1] and c2[1] > c1[1]:
                arr[i] = 'Iris-versicolor'
                cluster_Versicolor.append(c2[0])
            elif c3[1] > c2[1]:
                arr[i] = 'Iris-virginica'
                cluster_Virginica.append(c3[0]) 
        res_Setosa = []
        res_Versicolor = []
        res_Virginica = []
        for j in range(len(cluster_Setosa[0])):
            U_X = []
            for i in range(len(cluster_Setosa)):
                U_X.append(cluster_Setosa[i][j])
            res_Setosa.append(compute_cluster_center(m, U_X))
        clusters['Iris-setosa'] = res_Setosa

        for j in range(len(cluster_Versicolor[0])):
            U_X = []
            for i in range(len(cluster_Versicolor)):
                U_X.append(cluster_Versicolor[i][j])
            res_Versicolor.append(compute_cluster_center(m, U_X))
        clusters['Iris-versicolor'] = res_Versicolor
        
        for j in range(len(cluster_Virginica[0])):
            U_X = []
            for i in range(len(cluster_Virginica)):
                U_X.append(cluster_Virginica[i][j])
            res_Virginica.append(compute_cluster_center(m, U_X))
        clusters['Iris-virginica'] = res_Virginica
        
        if clusters['Iris-setosa'] == prev['Iris-setosa'] and clusters['Iris-versicolor'] == prev['Iris-versicolor'] and clusters['Iris-virginica'] == prev['Iris-virginica']:
            print('Converged before Max iter i.e., no of iter:', epoch)
            return arr
        prev = {'Iris-setosa': clusters['Iris-setosa'], 'Iris-versicolor': clusters['Iris-versicolor'], 'Iris-virginica': clusters['Iris-virginica']}
        epoch += 1
    print('Max iter:', epoch-1)
    return arr



In [4]:
def remove_index(dataset):
    brr = []
    for i in dataset:
        brr.append(i.pop())
    return [dataset, brr]


def load_csv(filename):
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            dataset.append(row)
    dataset.pop(0)
    return dataset

In [ ]:
def main():
    k = 3
    m = 2
    filename = 'Datasets/Iriscmean.csv'
    dataset = load_csv(filename)
    random.shuffle(dataset)
    clusters = {}
    res = []
    res1 = random.sample(dataset, 3)
    for i in res1:
        res.append(list(i))
    res[0].pop(0)
    res[1].pop(0)
    res[2].pop(0)
    for i in range(len(res[0])-1):
        res[0][i] = float(res[0][i].strip())
        res[1][i] = float(res[1][i].strip())
        res[2][i] = float(res[2][i].strip())
    clusters = {'Iris-setosa': res[0], 'Iris-versicolor': res[1], 'Iris-virginica': res[2]}
    crr = remove_index(dataset)
    dataset = crr[0]
    for i in range(len(dataset[0])-1):
        for row in dataset:
            row[i] = float(row[i].strip())
    brr = crr[1]
    print("Actual no of tuples belongs to 'iris-setosa' class:", brr.count('Iris-setosa'))
    print("Actual no of tuples belongs to 'iris-virginica' class:", brr.count('Iris-virginica'))
    print("Actual no of tuples belongs to 'iris-versicolor' class:", brr.count('Iris-versicolor'))
    final_cluster = cMean(clusters, dataset, m)
    acc = 0
    for i in range(len(dataset)):
        if brr[i] == final_cluster[i]:
            acc += 1
    acc = acc/len(dataset)*100
    flag = True
    if acc < 70:
        flag = False

    if flag == False:
        for i in range(len(final_cluster)):
            if final_cluster[i] == 'Iris-setosa':
                final_cluster[i] = 'Iris-versicolor'
            elif final_cluster[i] == 'Iris-versicolor':
                final_cluster[i] = 'Iris-virginica'
            elif final_cluster[i] == 'Iris-virginica':
                final_cluster[i] = 'Iris-setosa'
    for i in range(len(dataset)):
        if brr[i] == final_cluster[i]:
            acc += 1
    acc = acc/len(dataset)*100
    print("After clustering no of tuples belongs to 'Iris-setosa' class:",
          final_cluster.count('Iris-setosa'))
    print("After clustering no of tuples belongs to 'Iris-versicolor' class:",
          final_cluster.count('Iris-versicolor'))
    print("After clustering no of tuples belongs to 'Iris-virginica' class:",
          final_cluster.count('Iris-virginica'))
    pre = TrueP(brr, final_cluster) / \
        (TrueP(brr, final_cluster)+FalseP(brr, final_cluster))
    recall = TrueP(brr, final_cluster) / \
        (TrueP(brr, final_cluster)+FalseN(brr, final_cluster))
    print("Accuracy:", acc)
    print("Precision:", pre*100)
    print("Recall:", recall*100)

    print("Confusion matrix:")
    print("TrueP=", TrueP(brr, final_cluster))
    print("TrueN=", TrueN(brr, final_cluster))
    print("FalseP=", FalseP(brr, final_cluster))
    print("FalseN=", FalseN(brr, final_cluster))
    print()
    class_names = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    ret_array = confusion_matrix(brr, final_cluster, labels=class_names)
    plot_confusion_matrix(ret_array, class_names, normalize=False,
                          title='Confusion matrix', cmap=plt.cm.Blues)


if __name__ == '__main__':
    main()


Actual no of tuples belongs to 'iris-setosa' class: 50
Actual no of tuples belongs to 'iris-virginica' class: 50
Actual no of tuples belongs to 'iris-versicolor' class: 50
